# Рекомендация мобильного тарифа

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разбейте данные на выборки

Разделим данные на обучающую, валидационнуюи тренировочную выборки.
Вначале разделим на обучающую и валидационную, а затем валидационную разделим пополам на валидационную и тренировочную.

In [4]:
df_train, df_valid = train_test_split(df, train_size=0.60, test_size=0.40, random_state=17)

In [5]:
df_valid, df_test = train_test_split(df_valid, train_size=0.50, test_size=0.50, random_state=17)

В каждой выборке выделяем признаки и целевой признак.

In [6]:
# обучающая выборка
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']

# валидационная выборка
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

# тестовая выборка
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [20]:
print('Размеры выборок \nОбучающей:',
      features_train.shape,
      '\nВалидационной:',
      features_valid.shape,
      '\nТестовой:',
      features_test.shape)

Размеры выборок 
Обучающей: (1928, 4) 
Валидационной: (643, 4) 
Тестовой: (643, 4)


## Исследуйте модели

### Решающее древо

In [7]:
best_model_tree = None
best_result_tree = 0

for depth in range(1,5):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_tree = model_tree.predict(features_valid)
    accuracy_tree = accuracy_score(target_valid, predictions_tree)
    if accuracy_tree > best_result_tree:
        best_result_tree = accuracy_tree
        best_model_tree = model_tree
        depth_p = depth
        
print("Accuracy модели древа на валидационной выборке:",accuracy_tree)
print('Глубина древа:', depth_p)

Accuracy модели древа на валидационной выборке: 0.7993779160186625
Глубина древа: 3


### Случайный лес

In [8]:
best_model_forest = None
best_result_forest = 0

for est in range(1,10):
    for depth in range(1,5):
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth = depth)
        model_forest.fit(features_train, target_train)
        predictions_forest = model_forest.predict(features_valid)
        accuracy_forest = accuracy_score(target_valid, predictions_forest)
        if accuracy_forest > best_result_forest:
            best_result_forest = accuracy_forest
            best_model_forest = model_forest
            est_forest = est
            depth_f = depth
            
print("Accuracy модели случайного леса на валидационной выборке:",accuracy_forest)
print('Наилучшее количество деревьев:', est_forest)
print('Лучшая глубина древа:', depth_f)

Accuracy модели случайного леса на валидационной выборке: 0.807153965785381
Наилучшее количество деревьев: 8
Лучшая глубина древа: 3


### Логистическая регрессия

In [9]:
model_reg = LogisticRegression(random_state=12345)

model_reg.fit(features_train, target_train)

predictions_reg = model_reg.predict(features_valid)

accuracy_reg = accuracy_score(target_valid, predictions_reg)

print("Accuracy модели регрессии на валидационной выборке:",accuracy_reg)

Accuracy модели регрессии на валидационной выборке: 0.76049766718507


Из проверенных на валидационной выборке моделях можно выбрать модель случайного леса у нее лучшие показатели.

## Проверьте модель на тестовой выборке

In [10]:
def test_model(model):
    predictions = model.predict(features_test)
    accuracy = accuracy_score(target_test, predictions)
    return accuracy

In [11]:
print("Accuracy лучшей модели случайного леса на тестовой выборке:",test_model(best_model_forest))

Accuracy лучшей модели случайного леса на тестовой выборке: 0.7822706065318819


## (бонус) Проверьте модели на адекватность

Для проверки модели на адекватность воспользуемся классификатором 'DummyClassifier'.

In [12]:
strategies = ['stratified', 'most_frequent', 'prior', 'uniform'] 

best_strat = None
best_result_strat = 0

for strat in strategies:
    dm_cl = DummyClassifier(strategy = strat)
    dm_cl.fit(features_train, target_train)
    predictions_dm = dm_cl.predict(features_test)
    result = accuracy_score(target_test, predictions_dm)
    if result > best_result_strat:
            best_result_strat = result
            best_strat = strat
            
print('Accuracy лучшей стратегии:', best_result_strat)
print('Наилучшая стратегия:', best_strat)            

Accuracy лучшей стратегии: 0.6982892690513219
Наилучшая стратегия: most_frequent


Показатель 'Accuracy' выбраной модели случайного леса выше (0.78), чем у стратегии 'most_frequent' классификатора 'DummyClassifier' (0.70).

## Вывод

Проверка модели на тестовой выборке показала, что выбранная модель случайного леса работает достаточно хорошо. Она способна правильно предсказать около 78% ответов.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
